# 网络模块

## socket模块

套接字是模块socket中socket类的实例。实例化套接字时最多可指定三个参数：一个地址族（默认为socket.AF_INET）；是流套接字（socket.SOCK_STREAM，默认设置）还是数据报套接字（socket.SOCK_DGRAM）；协议（使用默认值0就好）。创建普通套接字时，不用提供任何参数。

服务器套接字先调用方法bind，再调用方法listen来监听特定的地址。然后，客户端套接字就可连接到服务器了，办法是调用方法connect并提供调用方法bind时指定的地址（在服务器端，可使用函数socket.gethostname获取当前机器的主机名）。这里的地址是一个格式为(host, port)的元组，其中host是主机名（如www.example.com），而port是端口号（一个整数）。方法listen接受一个参数——待办任务清单的长度（即最多可有多少个连接在队列中等待接纳，到达这个数量后将开始拒绝连接）。

服务器套接字开始监听后，就可接受客户端连接了，这是使用方法accept来完成的。这个方法将阻断（等待）到客户端连接到来为止，然后返回一个格式为(client, address)的元组，其中client是一个客户端套接字，而address是前面解释过的地址。服务器能以其认为合适的方式处理客户端连接，然后再次调用accept以接着等待新连接到来。这通常是在一个无限循环中完成的。

“注意　这里讨论的服务器编程形式称为阻断（同步）网络编程。”

为传输数据，套接字提供了两个方法：send和recv（表示receive）。要发送数据，可调用方法send并提供一个字符串；要接收数据，可调用recv并指定最多接收多少个字节的数据。如果不确定该指定什么数字，1024是个不错的选择。

In [ ]:
## 简单的服务器

import socket
s= socket.socket()

host = socket.gethostname()
port = 1234
s.bind((host,port))

s.listen(5)
while True:
    c,addr = s.accept()
    print('Got connection from',addr)
    c.send(b'Thank you for connecting')
    c.close()

In [ ]:
## 简单的客户端
import socket
s= socket.socket()
host = socket.gethostname()
port = 1234
s.connect((host,port))
print(s.recv(1024))

In [4]:
# “urlopen返回的类似于文件的对象支持方法close、read、readline和readlines，还支持迭代等。
from urllib.request import urlopen
webpage = urlopen('http://www.python.org')
webpage

In [5]:
import re
text = webpage.read()
m = re.search(b'<a href="([^"]+)".*?>about</a>',text,re.IGNORECASE)
print(m.group())
print(m.group(1))

b'<a href="/about/" title="" class="">About</a>'
b'/about/'


In [8]:
from urllib.request import urlretrieve
urlretrieve('https://www.python.org','/Users/soft98/Documents/git/websec/Language/python-study/jupyter-file/py-file/14.1-urlretrieve')


('/Users/soft98/Documents/git/websec/Language/python-study/jupyter-file/py-file/14.1-urlretrieve',
 <http.client.HTTPMessage at 0x7fc97e641748>)

+ quote(string[, safe])：返回一个字符串，其中所有的特殊字符（在URL中有特殊意义的字符）都已替换为对URL友好的版本（如将~替换为%7E）。如果要将包含特殊字符的字符串用作URL，这很有用。参数safe是一个字符串（默认为'/'），包含不应像这样对其进行编码的字符。
+ quote_plus(string[, safe])：类似于quote，但也将空格替换为加号。
+ unquote(string)：与quote相反。
+ unquote_plus(string)：与quote_plus相反。
+ urlencode(query[, doseq])：将映射（如字典）或由包含两个元素的元组（形如(key, value)）组成的序列转换为“使用URL编码的”字符串。这样的字符串可用于CGI查询中（详细信息请参阅Python文档）。

Cookie
Cookie对象操作，主要用于服务器 

cookielib
客户端Cookie支持

httplib
HTTP 客户端模块

robotparser
解析Web服务器robot文件

SimpleXMLRPCServer
一个简单的XML-RPC服务器（参见第27章）

smtpd
SMTP服务器模块

smtplib
SMTP客户端模块

telnetlib
Telnet客户端模块

urlparse
用于解读URL

xmlrpclib
XML-RPC客户端支持（参见第27章）

编写简单的套接字服务器并不难。然而，如果要创建的并非简单服务器，还是求助于服务器模块吧。模块SocketServer是标准库提供的服务器框架的基石，这个框架包括BaseHTTPServer、SimpleHTTPServer、CGIHTTPServer、SimpleXMLRPCServer和DocXMLRPCServer等服务器，它们在基本服务器的基础上添加了各种功能。

SocketServer包含4个基本的服务器：TCPServer（支持TCP套接字流）、UDPServer（支持UDP数据报套接字）以及更难懂的UnixStreamServer和UnixDatagramServer。后面3个你可能不会用到。

使用模块SocketServer编写服务器时，大部分代码都位于请求处理器中。每当服务器收到客户端的连接请求时，都将实例化一个请求处理程序，并对其调用各种处理方法来处理请求。具体调用哪些方法取决于使用的服务器类和请求处理程序类；还可从这些请求处理器类派生出子类，从而让服务器调用一组自定义的处理方法。基本请求处理程序类BaseRequestHandler将所有操作都放在一个方法中——服务器调用的方法handle。这个方法可通过属性self.request来访问客户端套接字。如果处理的是流（使用TCPServer时很可能如此），可使“用StreamRequestHandler类，它包含另外两个属性：self.rfile（用于读取）和self.wfile（用于写入）。你可使用这两个类似于文件的对象来与客户端通信。

In [ ]:
# 基于SocketServer的极简服务器
from socketserver import TCPServer, StreamRequestHandler

class Handler(StreamRequestHandler):
    def handle(self):
        addr = self.request.getpeername()
        print('Got connectio from', addr)
        self.wfile.write('Thank you for connecting')

server = TCPServer(('',1234),Handler)
server.serve_forever()

处理多个连接的主要方式有三种：分叉（forking）、线程化和异步I/O。通过结合使用SocketServer中的混合类和服务器类，很容易实现分叉和线程化

In [ ]:
# 分叉服务器
from socketserver import TCPServer,ForkingMixIn,StreamRequestHandler

class Server(ForkingMixIn,TCPServer):pass
class Handler(StreamRequestHandler):
    def handle(self):
        addr = self.request.getpeername()
        print('Got connection from',addr)
        self.wfile.write(b'Thank you for connecting')
server = Server(('',1234),Handler)
server.serve_forever()

In [ ]:
# 线程服务器
from socketserver import TCPServer,ThreadingMixIn,StreamRequestHandler

class Server(ThreadingMixIn,TCPServer):pass
class Handler(StreamRequestHandler):
    def handle(self):
        addr = self.request.getpeername()
        print('Got connection from',addr)
        self.wfile.write(b'Thank you for connecting')
        
server = Server(('',1234),Handler)
server.serve_forever()

函数select接受三个必不可少的参数和一个可选参数，其中前三个参数为序列，而第四个参数为超时时间（单位为秒）。这些序列包含文件描述符整数（也可以是这样的对象：包含返回文件描述符整数的方法fileno），表示我们正在等待的连接。这三个序列分别表示需要输入和输出以及发生异常（错误等）的连接。如果没有指定超时时间，select将阻断（即等待）到有文件描述符准备就绪；如果指定了超时时间，select将最多阻断指定的秒数；如果超时时间为零，select将不断轮询（即不阻断）。select返回三个序列（即一个长度为3的元组），其中每个序列都包含相应参数中处于活动状态的文件描述符。例如，返回的第一个序列包含有数据需要读取的所有输入文件描述符。这些序列也可包含文件对象（Windows不支持）或套接字。

In [ ]:
# 使用select的简单服务器
import socbket,select

s = socket.socket()
host = socket.gethostname()
port = 1234
s.bind((host,port))
s.listen(5)
inputs = [s]
while True:
    rs,ws,es = select.select(inputs,[],[])
    for r in rs:
        if r is s:
            c,addr = s.accept()
            print('Got connection from',addr)
            inputs.append(c)
        else:
            try:
                data = r.recv(1024)
                disconnected = not data
            except socket.error:
                disconnected = True
            
            if disconnected:
                print(r.getpeername(),'disconnected')
                inputs.remove(r)
            else:
                print(data)

方法poll使用起来比select容易。调用poll时，将返回一个轮询对象。你可使用方法register向这个对象注册文件描述符（或包含方法fileno的对象）。注册后可使用方法unregister将它们删除。注册对象（如套接字）后，可调用其方法poll（它接受一个可选的超时时间参数）。这将返回一个包含(fd, event)元组的列表（可能为空），其中fd为文件描述符，而event是发生的事件。event是一个位掩码，这意味着它是一个整数，其各个位对应于不同的事件。各种事件是用select模块中的常量表示的，如表14-2所示。要检查指定的位是否为1（即是否发生了相应的事件），可下面这样使用按位与运算符（&）：

if event & select.POLLIN: ...

select模块中的轮询事件常量

POLLIN
文件描述符中有需要读取的数据

POLLPRI
文件描述符中有需要读取的紧急数据

POLLOUT
文件描述符为写入数据做好了准备

POLLERR
文件描述符出现了错误状态

POLLHUP
挂起。连接已断开。

POLLNVAL
无效请求。连接未打开

In [ ]:
# 使用poll的简单服务器

import socket,select
s = socket.socket()
host = socket.gethostname()
port = 1234
s.bind((host,port))

fdmap = {s.fileno():s}

s.listen(5)
p = select.poll()
p.register(s)
while True:
    events = p.poll()
    for fd,event in events:
        if fd in fdmap:
            c,addr = s.accept()
            print('Got connection from',addr)
            p.register(c)
            fdmap[c.fileno()] = c
        elif event & select.POLLIN:
            data = fdmap[fd].recv(1024)
            if not data:
                print(fdmap[fd].getpeername(),'disconnected')
                p.unregister(fd)
                del fdmap[fd]
            else:
                print(data)

Twisted是由Twisted Matrix Laboratories http://twistedmatrix.com 开发的，这是一个事件驱动的Python网络框架，最初是为编写网络游戏开发的，但现被各种网络软件使用。在Twisted中，你能实现事件处理程序，就像在GUI工具包（参见第12章）中一样。实际上，Twisted与多个常用的GUI工具包（Tk、GTK、Qt和wxWidgets）配合得天衣无缝。

In [ ]:
# Twisted 简单服务器
from twisted.internet import reactor
from twisted.internet.protocol import Protocol, Factory

class SimpleLogger(Protocol):

    def connectionMade(self):
        print('Got connection from', self.transport.client)

    def connectionLost(self, reason):
        print(self.transport.client, 'disconnected')

    def dataReceived(self, data):
        print(data)

factory = Factory()
factory.protocol = SimpleLogger
reactor.listenTCP(1234, factory)
reactor.run()

In [ ]:
# 使用LineReceiver改进后的日志服务器
from twisted.internet import reactor
from twisted.internet.protocol import Factory
from twisted.protocols.basic import LineReceiver

class SimpleLogger(LineReceiver):

    def connectionMade(self):
        print('Got connection from', self.transport.client)

    def connectionLost(self, reason):
        print(self.transport.client, 'disconnected')

    def lineReceived(self, line):
        print(line)

factory = Factory()
factory.protocol = SimpleLogger

reactor.listenTCP(1234, factory)
reactor.run()